# SS記事の自動カテゴリ分類

運営しているSSまとめサイトで、それぞれのSSのカテゴリ付けを自動化できないか検証してみた。

使用したデータは、DBにクエリを投げてダンプしたもの。

このノートブックでは、それぞれのカテゴリにおいてTF-IDF値が大きい単語を特徴量として分類する。

### 環境

- Ubuntu 14.04
- python3.5

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from joblib import Parallel, delayed
import multiprocessing
from collections import Counter
import re

from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

In [2]:
# pandas.DataFrame.apply関数の並列版
def applyParallel(dfGrouped, func, asDataFrame=False):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(field) for field in dfGrouped)
    if asDataFrame:
        return pd.DataFrame(retLst)
    else:
        return pd.Series(retLst)

## データ理解

データの読み込み

In [3]:
# セパレータがカンマだとうまく読み込めなかったので、独自セパレータを使用
df = pd.read_csv('./articles.csv', sep='PNDFAKFD', encoding='utf-8')

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


In [4]:
df[:10]

,article_id,category_id,category_name,html
0,1,3.0,艦これ,<dd>提督「(俺はこの鎮守府の提督)」 <br> <br> 提督「(深海棲艦という、謎の...
1,2,3.0,艦これ,<dd>時雨「今日は雨だね、提督」 <br> <br> 時雨「そういえば」 <br> <...
2,3,NaN,NaN,<dd>女騎士「何！？オークとオーガは似たような種族ではないのか！？」 <br> <br>...
3,4,2.0,男女,<dd> 幼「約束は８時でしょ？」<br><br>男「悪かったよ、ちょっと仕事がな…」<br...
4,5,11.0,化物語,<dd> <br> ・アイドルマスター・ディアリースターズと物語シリーズのクロスです。 <b...
5,6,NaN,NaN,<dd> <br> メリオダス・ギル坊「じゃんけんほい！！！」 <br> <br> <b...
6,7,NaN,NaN,<dd> <br> <br> <br> <br> <br> ｻﾞｧｰｰｰｰｰｰｰ･･...
7,8,10.0,モバマス,<dd>秋風温泉ガチャをネタ元にしたＳＳです <br> 内容はぶっちゃけ蘭子かな子芳乃とのハ...
8,9,10.0,モバマス,<dd>冬ということは忘れてください <br> <br> <br> <br> 「「「た...
9,10,NaN,NaN,"<dd class=""net""> 俺「よろしく！うわぁー、ワクワクするなぁ！」 <br> <..."


カテゴリIDとカテゴリ名の対応付け

In [5]:
tmp_df = df[['category_id', 'category_name']].dropna()
tmp_df.category_id = tmp_df.category_id.astype(int)

category_df = tmp_df.groupby(['category_id']).agg(lambda x:x.value_counts().index[0])
category_df[:5]

,category_name
category_id,
1,俺妹
2,男女
3,艦これ
4,とある魔術の禁書目録
5,兄妹・姉弟


In [6]:
count_each_category = df[['article_id', 'category_name']].groupby(['category_name']).agg('count').sort_values('article_id', ascending=False)
count_each_category[:15]

,article_id
category_name,
モバマス,4586
艦これ,1787
アイドルマスター,1642
俺ガイル,953
ラブライブ！,725
男女,695
シュタインズ・ゲート,461
勇者・魔王,388
咲,308


モバマス・艦これが多い

## データ整形

まずは記事数が多くて、かつ分類しやすそうなカテゴリだけ選んで試してみる。

(ex. 「男女」と「兄妹・姉弟」などは分類しにくそうなので除外）

In [7]:
easy_categories = ['モバマス', '艦これ', 'アイドルマスター', '俺ガイル', 'ラブライブ！', 'シュタインズ・ゲート', '咲', 'とある魔術の禁書目録', 'ゆるゆり', 'ガールズ＆パンツァー']
easy_df = df[df.category_name.isin(easy_categories)]

print('カテゴリー数: {}'.format(len(easy_categories)))
print('対象記事数: {}'.format(easy_df.shape[0]))

カテゴリー数: 10
対象記事数: 11339


BeautifulSoupでhtmlタグとurlを除去。
8コアをフルに使っても2分くらいかかる。

In [8]:
def extract_text(html):
    text = BeautifulSoup(html, "html.parser").getText() # htmlタグを除去
    return re.sub(r'https?://[^\s/$.?#].[^\s]*', '', text) # urlを除去

easy_df['text'] = applyParallel(easy_df.html, extract_text).values

easy_df = easy_df.drop('html', axis=1)
easy_df.category_id = easy_df.category_id.astype(int)
easy_df[:10]

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,article_id,category_id,category_name,text
0,1,3,艦これ,提督「(俺はこの鎮守府の提督)」 提督「(深海棲艦という、謎の敵から海を守るため)」 ...
1,2,3,艦これ,時雨「今日は雨だね、提督」 時雨「そういえば」 時雨「レイテ沖のときも雨が降っていた...
7,8,10,モバマス,秋風温泉ガチャをネタ元にしたＳＳです 内容はぶっちゃけ蘭子かな子芳乃とのハーレムエロなんで...
8,9,10,モバマス,冬ということは忘れてください 「「「ただいまー！！」」」 P「おう みんなおかえ...
11,12,3,艦これ,提督「珍しいな、溜息なんてついて」 那珂「あっ、提督！ たたた溜息なんてついてないよ！？...
12,13,9,アイドルマスター,＿＿＿＿＿＿＿＿＿＿＿＿７６５プロ 事務所 仕事終わりの事務所にやよい、真、響、春香、美...
15,16,3,艦これ,【加賀の場合】 「提督、失礼します」 「あら……寝ているの？」 「最近忙しか...
18,19,3,艦これ,提督の第一印象？ ……提督だとは思わなかった、ですね。 ふざけた芸人...
19,20,3,艦これ,提督「(雲龍がこの鎮守府に来てくれて)」 提督「(しばらく経った)」 提督「(今では...
20,21,9,アイドルマスター,"""5"" ""4"" 千早「プロデューサー、いよいよですね」 ""3"" ..."


## 特徴量に使う単語辞書作成

それぞれのカテゴリから、td-idf値の高い単語を抽出する。

In [9]:
import MeCab
tagger = MeCab.Tagger('-Ochasen -d /usr/lib/mecab/dic/mecab-ipadic-neologd')
tagger.parse('') #　おまじない(mecabのバグ)

'EOS\n'

Nelogd辞書がちゃんと機能しているかチェック

In [10]:
node = tagger.parseToNode("新辞書が必要なのは、なのはみたいなややこしい名前が入ってるとき")
node = node.next

while node and node.surface:
    print("{}\t\t{}".format(node.surface, node.feature))
    node = node.next

新		接頭詞,名詞接続,*,*,*,*,新,シン,シン
辞書		名詞,一般,*,*,*,*,辞書,ジショ,ジショ
が		助詞,格助詞,一般,*,*,*,が,ガ,ガ
必要		名詞,形容動詞語幹,*,*,*,*,必要,ヒツヨウ,ヒツヨー
な		助動詞,*,*,*,特殊・ダ,体言接続,だ,ナ,ナ
の		名詞,非自立,一般,*,*,*,の,ノ,ノ
は		助詞,係助詞,*,*,*,*,は,ハ,ワ
、		記号,読点,*,*,*,*,、,、,、
なのは		名詞,固有名詞,人名,一般,*,*,なのは,ナノハ,ナノハ
みたい		名詞,非自立,形容動詞語幹,*,*,*,みたい,ミタイ,ミタイ
な		助動詞,*,*,*,特殊・ダ,体言接続,だ,ナ,ナ
ややこしい		形容詞,自立,*,*,形容詞・イ段,基本形,ややこしい,ヤヤコシイ,ヤヤコシイ
名前		名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
が		助詞,格助詞,一般,*,*,*,が,ガ,ガ
入っ		動詞,自立,*,*,五段・ラ行,連用タ接続,入る,ハイッ,ハイッ
てる		動詞,非自立,*,*,一段,基本形,てる,テル,テル
とき		名詞,非自立,副詞可能,*,*,*,とき,トキ,トキ


一般名詞と固有名詞だけ抽出

In [11]:
def get_separted_words(text):
    word_array = []
    
    node = tagger.parseToNode(text)
    while node:
        split = node.feature.split(',')
        if split[0] == '名詞' and (split[1] in ['一般', '固有名詞']):
            word_array.append(node.surface)
        node = node.next
    return word_array

easy_df['words'] = applyParallel(easy_df.text, get_separted_words).values
easy_df[:5][['category_name', 'words']]

,category_name,words
0,艦これ,"[提督, 鎮守府, 提督, 提督, 深海, 棲艦, 謎, 敵, 海, 提督, 艦娘, 一緒に..."
1,艦これ,"[時雨, 今日は雨, 提督, 時雨, 時雨, レイテ, 沖, 雨, 時雨, 時雨, 扶桑, ..."
7,モバマス,"[秋風, 温泉, ガチャ, ネタ, ＳＳ, 内容, ぶっちゃけ, 蘭子, かな子, 芳, 乃..."
8,モバマス,"[冬, ー, P, 薫, ぇ, P, 薫, 薫, ザリガニ, ザリガニ, ｸﾜｧ, P, 千..."
11,艦これ,"[提督, 溜息, 那珂, 提督, 溜息, 那珂, ア, イドル, 提督, 那珂, アイドル,..."


カテゴリごとにwordsを結合してコーパスを作る。（並列化できてないので重い）

In [12]:
grouped_words = easy_df[['category_name', 'words']].groupby(['category_name']).agg('sum')
grouped_words

,words
category_name,
とある魔術の禁書目録,"[上条, ω, 前回, 上条, 上条, キャラ, >>1, キャラ, とこ, SSWiki,..."
ゆるゆり,"[あかり, 不満, 赤座あかり, 空, 空, 世界, 赤座あかり, 暇, 空, 空, 飛行機..."
アイドルマスター,"[プロ, 事務所, 事務所, やよい, 真, 響, 春香, 美希, プロデューサー, 机, ..."
ガールズ＆パンツァー,"[沙織, 彼氏, 彼氏, 優花里, 彼氏, どうでしょう, カチューシャ, 尻相撲, 優花里..."
シュタインズ・ゲート,"[未来, ガジェット, 研究所, 岡部, フゥーハハハ, 狂気, マァッドサイエンティスト,..."
モバマス,"[秋風, 温泉, ガチャ, ネタ, ＳＳ, 内容, ぶっちゃけ, 蘭子, かな子, 芳, 乃..."
ラブライブ！,"[凛, 姫ちゃん, 姫, 花陽, 姫, 先, 凛, 花陽, 姫, ごめんね, 凛, 次, 花..."
俺ガイル,"[葉山, 比企, 谷, 葉山, 陽, 乃, ドリンク, 葉山, 全身, 痛み, 葉山, 子供..."
咲,"[ギバ, ぢ, ゃ゛ん゛が, テレビ, こ, ア, ア, 咏, 自分, 全国, いって, ギ..."


カテゴリごとに、tf-idfが高い単語を抽出する。

In [35]:
counters = list(map(lambda word_array: Counter(word_array), grouped_words.words.values))

cutoff = 0.001
def get_tf_idf_above_cutoff(counter):
    global counters, cutoff
    
    words = counter.most_common(200) # [('hello', 142), ('world', 24), ...]
    word_len = sum(counter.values())
    
    tf_idf = Counter()
    for word in words:
        tf = word[1] / word_len
        idf = np.log(len(counters) / sum(map(lambda counter: 1 if counter[word[0]] > 0 else 0, counters)))
        if tf * idf > cutoff:
            tf_idf[word[0]] = tf * idf
    return tf_idf

high_tfidf_words = list(map(get_tf_idf_above_cutoff, counters))
list(map(lambda x: x.most_common(3), high_tfidf_words)) # それぞれ上位3つのみ表示

[[('ミサカ', 0.02055882485008614),
  ('美琴', 0.019454978025012542),
  ('上条', 0.019116087569487437)],
 [('櫻子', 0.054716933276526714),
  ('京子', 0.052083736946399381),
  ('あかり', 0.04762419467506071)],
 [('伊織', 0.013966713510533444),
  ('春香', 0.012453572113815608),
  ('美希', 0.010367820419800272)],
 [('エリカ', 0.044051734611033753),
  ('優花里', 0.023809148664592508),
  ('沙織', 0.016377129543714931)],
 [('岡部', 0.11048434720109576),
  ('栖', 0.041943341691853014),
  ('郁', 0.02017804786283945)],
 [('モバ', 0.014257525329247822),
  ('幸子', 0.009183117912948428),
  ('菜々', 0.0091188290413308382)],
 [('花陽', 0.060050368850414031),
  ('絵里', 0.035700395931709462),
  ('果', 0.03336997107730158)],
 [('八幡', 0.11144398691385726),
  ('由比ヶ浜', 0.023345991448346943),
  ('雪ノ下', 0.020787907239110934)],
 [('京太郎', 0.043389555367994589),
  ('竜華', 0.022630988277614717),
  ('憧', 0.021505790785062904)],
 [('提督', 0.022535396162667077),
  ('加賀', 0.016472915047054462),
  ('瑞鶴', 0.013695844511141709)]]

かなり良い感じの単語を抽出出来ているようだ。

単語 -> インデックスのdictionaryを作成

In [36]:
def flatten(l):
    return [item for sublist in l for item in sublist]

all_words = set(flatten(map(lambda counter: counter.keys(), high_tfidf_words)))

word2index = {}
for i,v in enumerate(all_words):
    word2index[v] = i

num_words = len(word2index)
"辞書内の単語数: {}".format(num_words)

'辞書内の単語数: 484'

それぞれの記事から、辞書内の単語のみ抽出

8コアで2分くらいかかる

In [37]:
def get_feature_words(text):
    word_array = get_separted_words(text)
    feature_word_array = []
    for word in word_array:
        if word in word2index:
            feature_word_array.append(word)
    return feature_word_array

easy_df['feature_words'] = applyParallel(easy_df.text, get_feature_words).values
easy_df[:10][['category_name', 'feature_words']]

,category_name,feature_words
0,艦これ,"[提督, 鎮守府, 提督, 提督, 棲艦, 提督, 艦娘, 提督, 艦娘, 提督, 提督, ..."
1,艦これ,"[時雨, 提督, 時雨, 時雨, 時雨, 時雨, 扶桑, 山城, 時雨, 満潮, 時雨, 時..."
7,モバマス,"[蘭子, かな子, 芳, 乃, 蘭子, かな子, 芳, 乃, 蘭子, かな子, 芳, 乃, ..."
8,モバマス,"[薫, 薫, 薫, 薫, 薫, 薫, 薫, 薫, 薫, 薫, 薫, 薫, 薫, 薫, 薫, ..."
11,艦これ,"[提督, 那珂, 提督, 那珂, 提督, 那珂, 提督, 那珂, 提督, 那珂, 提督, 那..."
12,アイドルマスター,"[やよい, 響, 春香, 美希, 響, 響, 響, 響, 美希, 春香, やよい, 美希, ..."
15,艦これ,"[加賀, 提督, 提督, 提督, 赤城さん, 提督, 艦娘, 戦艦, 青葉, 司令官, 司令..."
18,艦これ,"[提督, 提督, 提督, 提督, 提督, 赤城, 鎮守府, 提督, 提督, 提督, 赤城, ..."
19,艦これ,"[提督, 鎮守府, 提督, 提督, 提督, 提督, 艦娘, 提督, 提督, 提督, 提督, ..."
20,アイドルマスター,"[千早, 千早, 千早, 千早, 千早, 千早, 千早, 千早, 千早, 千早, 千早, 千..."


単語が出現した回数を考慮して、特徴量ベクトルに変換（長さが1になるように正規化）

In [38]:
def get_dense_features(feature_words):
    global word2index
    counter = Counter(feature_words)
    word_len = sum(counter.values())
    feature_vec = np.zeros(len(word2index), dtype=np.float64)
    
    for word in counter.items():
        # log(単語の出現回数 + 1) を特徴量とする
        feature_vec[word2index[word[0]]] = np.log(word[1] + 1)
    if np.linalg.norm(feature_vec) == 0:
        return feature_vec
    else:
        return feature_vec / np.linalg.norm(feature_vec) # n2ノルム正規化

easy_df[:3].feature_words.apply(get_dense_features)

0    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
7    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: feature_words, dtype: object

特徴量行列を作成

In [39]:
features = applyParallel(easy_df.feature_words, get_dense_features, asDataFrame=True)
labels = easy_df.category_id

## scikit-learnで予測

いくつかの手法で予測。

- SVM
- Logistic Regression
- Random Forest

データがそこそこ大きいので、解析的に解くモデルだと終わりません。

In [40]:
# テンプレート
def evaluate_model(model):
    global features, labels
    scores = cross_validation.cross_val_score(model, features, labels, cv=5)
    print("Accuracy: %0.5f (+/- %0.5f)" % (scores.mean(), scores.std() * 2))

### クロスバリデーションでモデルを評価

In [41]:
# SVM
evaluate_model(SGDClassifier(loss="hinge", penalty='l2', n_jobs=multiprocessing.cpu_count()))

Accuracy: 0.98668 (+/- 0.00463)


In [42]:
# ロジスティック回帰
evaluate_model(SGDClassifier(loss="log", penalty='l2', n_jobs=multiprocessing.cpu_count()))

Accuracy: 0.98324 (+/- 0.00434)


In [43]:
# ランダムフォレスト
evaluate_model(RandomForestClassifier(n_estimators=10, n_jobs=multiprocessing.cpu_count()))

Accuracy: 0.98254 (+/- 0.00509)


SVMが一番良い

以下SVMを使用

In [44]:
split_point = int(len(labels) * 0.8)
train_features = features[:split_point]
test_features = features[split_point:]
train_labels = labels[:split_point]
test_labels = labels[split_point:]

pred = SGDClassifier(loss="hinge", penalty='l2', n_jobs=multiprocessing.cpu_count()).fit(train_features, train_labels).predict(test_features)
pd.crosstab(pred, test_labels)

category_id,3,4,6,9,10,12,20,26,27,35
row_0,,,,,,,,,,
3,266,0,0,0,1,0,0,0,0,1
4,0,55,0,0,0,1,0,0,0,0
6,0,0,195,0,1,0,0,0,0,0
9,0,0,1,237,4,0,0,0,0,0
10,2,0,7,9,1079,0,0,0,1,1
12,0,0,0,0,0,95,0,0,0,0
20,0,0,0,0,0,0,61,0,0,0
26,0,0,0,0,0,0,0,51,0,0
27,0,0,0,1,0,0,0,0,40,0


分類に失敗したSSを眺めてみる

In [45]:
test_df = easy_df.ix[test_labels.index]
test_df['pred'] = category_df.category_name[pred].values

tmp = test_labels != pred
test_df.ix[tmp[tmp == True].index][['article_id', 'category_name', 'pred', 'feature_words']]

,article_id,category_name,pred,feature_words
15551,16187,アイドルマスター,モバマス,"[凛, モバ, 凛, 律子, 凛, モバ, 凛, モバ, 凛, モバ, 凛, モバ, モバ,..."
15629,16265,咲,モバマス,"[大星, 大星, 大星, 大星, 大星, 大星, 大星, 大星, 絹, 大星, 大星, 大星..."
15871,16507,モバマス,アイドルマスター,"[文香, 文香, ちひろ, 文香, ちひろ, ちひろ, 文香, ちひろ, 文香, 文香, ち..."
15873,16509,モバマス,ガールズ＆パンツァー,"[エリカ, エリカ, エリカ, 夕美, 夕美, 夕美, 夕美, 夕美]"
15996,16632,モバマス,艦これ,"[満潮, 鳥海, 秋津洲, 鎮守府, 駆逐艦, 提督, 提督, 提督, 司令官, 鎮守府, ..."
16120,16756,ラブライブ！,モバマス,[]
16144,16780,アイドルマスター,モバマス,[絵理]
16385,17021,シュタインズ・ゲート,とある魔術の禁書目録,"[上条当麻, 上条, 上条, 上条, 上条, 上条, 岡部, ダル, クリスティーナ, 上条..."
16634,17270,モバマス,アイドルマスター,"[雪歩, 雪歩]"
16777,17413,アイドルマスター,モバマス,"[茜, 茜, 茜, 茜, 茜, 茜, 茜, 茜, 茜, 茜, 茜, 茜, 茜, 茜, 美希,..."


## 考察

分類に失敗したものには、以下のパターンがあるようだ。（番号はarticle_id）

1. そもそも正解ラベルが間違っている（だいたいアイマス <-> モバマス）
  - 16632
  - 17270
  - 17413
  - 17462
  - 17689
  - 17770
  - 17901
  - 17925
  - 19018
  - 19022
  - 19308
2. 正解が曖昧（複数作品のクロスものなど）
  - 17021
  - 17501
3. 名前が似ているキャラクター
  - 16507
  - 16750
  - 17450
  - 18641
  - 18803
  - 19018
4. 台本形式じゃないため、名前がうまく取得できなかった
  - 16756
5. マイナーなキャラクターのSSだった
  - 16211
  - 16265
  - 16780
  - 17854
  - 18412
  - 18490
  - 18820
  - 19273
  - 19375
6. キャラクターの表記が本名じゃない
  - 18320
7. 形態素解析ができない名前のキャラ（にこ、真姫、穂乃果）
  - 18568
  - 19340

#### 思ったこと

- よくわからんものはモバマス、みたいな感じ
- 正解ラベル間違いすぎぃ！
- 3の理由のものは頑張って正しく分類できるようになりたい（2次の特徴量とか使えばいけるかも）
- 5,6は機械学習では無理かも
- 7はMecabの辞書次第

## まとめ

- だいたい98.7%くらいの正答率で分類できたよ！（人の方が精度はわずかに上）
- 正解ラベルが正しければ、99.2%くらいの精度かな？
- 特徴量tf-idf値の大きい名詞（基本的にはキャラクターの名前になる）を利用

## これから

- Mecabの辞書に少しキャラの名前足してみたい